In [16]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/TritangentPlane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [17]:
(sqrt(3)*I - sqrt(5)).minpoly()

x^4 - 4*x^2 + 64

In [18]:
alg_5 = 1/16*(sqrt(3)*I - sqrt(5))^3 - 3/4*(sqrt(3)*I - sqrt(5))

In [19]:
alg_5.expand()

sqrt(5)

In [20]:
# sqrt(5) = 1/16*i_rad3_minus_rad5^3 - 3/4*i_rad3_minus_rad5

In [21]:
alg_i = 1/16*(sqrt(3)*I - sqrt(5))^3 + 1/4*(sqrt(3)*I - sqrt(5))

In [22]:
alg_i.expand()

I*sqrt(3)

In [23]:
# i sqrt(3) =  1/16*i_rad3_minus_rad5^3 + 1/4*i_rad3_minus_rad5

In [24]:
xx = QQ['xx'].0

In [25]:
Q.<i_rad3_minus_rad5> = NumberField(xx^4 - 4*xx^2 + 64)

In [26]:
i_rad3 = 1/16*i_rad3_minus_rad5^3 + 1/4*i_rad3_minus_rad5
rad5 = 1/16*i_rad3_minus_rad5^3 - 3/4*i_rad3_minus_rad5

In [27]:
L_set_base = ['E1', 'G4', 'E2', 'G3', 'E3']

In [28]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(Q)
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [29]:
mon=((x+y+z+t)^3).monomials()

In [30]:
line = Line([y, z])
general_cubic = Cubic(cubic_new, line, sing_cubics)
SE1 = general_cubic.subs({b:-(c*c+e*f)/c})
SE2 = SE1.subs({d:(c*c+e*f)/c})
SE3 = SE1.subs({d:(3*e*f - c^2 + c*f + c*e)/(2*c)})
SE4 = SE3.subs({f:c*(3*c-e)/(c+e)})
SE6 = SE3.subs({f: -c*(5*c+e)/(c+e)})
SE9 = SE3.subs({e : i_rad3*c})
SE10 = SE4.subs({e : (2-rad5)*c})
SE18 = SE4.subs({e : i_rad3*c})

In [31]:
# Check that the three Eckardt points of SE3 are collinear
matrix([point for point in SE3.eckardt_points]).rank()

2

In [36]:
SE3.eckardt_points_labels

[['E1', 'G4', 'F14'], ['E2', 'G3', 'F23'], ['F13', 'F24', 'F56']]

In [37]:
# STUDY SUBFAMILIES

In [38]:
def get_unique_conditions(cubic):
    ideals = []
    gb = []
    table = []
    for pl in cubic.tritangent_planes:
        if pl.labels in [plane.labels for plane in cubic.tritangent_planes if plane.conditions==0]:
            continue
        poly = pl.conditions
        flag = True
        for ideal in ideals:
            if poly in ideal:
                table.append([pl.labels, gb.index(ideal.groebner_basis()[0])+1])
                flag = False
                break
        if flag == True:
            ideals.append(P.ideal(poly))
            gb.append(P.ideal(poly).groebner_basis()[0])
            table.append([pl.labels, gb.index(P.ideal(poly).groebner_basis()[0])+1])
    return [poly*poly.denominator() for poly in gb], table

In [39]:
# CONDITIONS FOR SE3

In [40]:
Q3, table3 = get_unique_conditions(SE3)

In [41]:
for i in range(1,len(Q3)+1):
    print(Q3[i-1], " ", len([el[0] for el in table3 if el[1]==i])+3)

5*c^2 - c*e - c*f + e*f   6
3*c^2 + c*e + c*f - e*f   4
c^2 + 3*c*e - c*f + e*f   6
c^2 - c*e + 3*c*f + e*f   6
5*c^2 + c*e + c*f + e*f   6
3*c^2 - c*e - c*f - e*f   4
c^2 + c*e - 3*c*f + e*f   6
c^2 - 3*c*e + c*f + e*f   6
c^2 + e*f   6
3*c^2 + e^2   9
3*c^2 + f^2   9
2*c - e + f   6
2*c + e - f   6
e + f   4


In [42]:
# SUBFAMILIES OF SE3 WITH 4 ECKARDT POINTS

In [43]:
SE3.eckardt_points_labels

[['E1', 'G4', 'F14'], ['E2', 'G3', 'F23'], ['F13', 'F24', 'F56']]

In [44]:
k3 = SE3.subs({f:-e})
k3.eckardt_points_labels

[['E1', 'G4', 'F14'],
 ['E2', 'G3', 'F23'],
 ['F13', 'F24', 'F56'],
 ['F14', 'F23', 'F56']]

In [45]:
k1 = SE3.subs({f:c*(3*c+e)/(e-c)})
k1.eckardt_points_labels

[['E1', 'G3', 'F13'],
 ['E1', 'G4', 'F14'],
 ['E2', 'G3', 'F23'],
 ['F13', 'F24', 'F56']]

In [46]:
SE4.eckardt_points_labels

[['E1', 'G4', 'F14'],
 ['E2', 'G3', 'F23'],
 ['E2', 'G4', 'F24'],
 ['F13', 'F24', 'F56']]

In [47]:
L_set_k3 = ('E1', 'F14', 'F23', 'G3', 'G1')
L_set_k1 = ('G4', 'E1', 'G3', 'E2', 'G2')

In [48]:
# Want to prove that the three families are projectively equivalent
# i.e. for any cubic S1 in k1, S2 in k2, S3 in k3
# there exist c1, e1, c3, e3 such that M^-1(S2) = S1(c1, e1) = S3(c3, e3)

In [49]:
proj_k3_SE4 = find_projectivity(k3.get_L_set_in_plucker(L_set_k3), SE4.get_L_set_in_plucker(L_set_base))
proj_k1_SE4 = find_projectivity(k1.get_L_set_in_plucker(L_set_k1), SE4.get_L_set_in_plucker(L_set_base))

In [50]:
# SUBFAMILIES OF SE3 WITH 6 ECKARDT POINTS

In [51]:
# SUBFAMILIES OF SE3 WITH 9 ECKARDT POINTS

In [52]:
u_2 = SE3.subs({e : -i_rad3*c})
L_set_SE9 = ['F46', 'G6', 'F26', 'F15', 'E3']

In [53]:
proj_SE9 = find_projectivity(SE9.get_L_set_in_plucker(L_set_SE9), SE9.get_L_set_in_plucker(L_set_base))
proj_SE9 = proj_SE9/(proj_SE9.list()[0].coefficients()[0])
proj_SE9 = proj_SE9*proj_SE9.denominator()
proj_SE9_inv = proj_SE9.inverse()
proj_SE9_inv = (proj_SE9_inv*proj_SE9_inv.denominator()).change_ring(P)

In [54]:
new_u_2 = u_2.subs({c:f-c, f: -c+2*i_rad3*c-f})
inverse_SE9 = SE9.subs(change_coord(proj_SE9_inv))

In [55]:
are_cubics_same(new_u_2, inverse_SE9)

True

In [56]:
are_cubics_same(new_u_2.subs(change_coord(proj_SE9)), SE9)

True

In [57]:
# CONDITIONS FOR SE4

In [58]:
Q4, table4 = get_unique_conditions(SE4)

In [59]:
for i in range(1,len(Q4)+1):
    print(Q4[i-1].factor(), " ", len([el[0] for el in table4 if el[1]==i])+4)
    print()

(c + (1/16*i_rad3_minus_rad5^3 - 3/4*i_rad3_minus_rad5 + 2)*e) * (c + (-1/16*i_rad3_minus_rad5^3 + 3/4*i_rad3_minus_rad5 + 2)*e)   10

(3) * (c + (1/48*i_rad3_minus_rad5^3 + 1/12*i_rad3_minus_rad5)*e) * (c + (-1/48*i_rad3_minus_rad5^3 - 1/12*i_rad3_minus_rad5)*e)   18

(c + (1/16*i_rad3_minus_rad5^3 - 3/4*i_rad3_minus_rad5 - 2)*e) * (c + (-1/16*i_rad3_minus_rad5^3 + 3/4*i_rad3_minus_rad5 - 2)*e)   10

(5) * (c + (1/80*i_rad3_minus_rad5^3 - 3/20*i_rad3_minus_rad5)*e) * (c + (-1/80*i_rad3_minus_rad5^3 + 3/20*i_rad3_minus_rad5)*e)   10

1   13



In [60]:
[[solve_linear_system_in_fraction_field([q[0]], [e], [c]) for q in Q4[i].factor()] for i in range(4)]

[[[(-1/16*i_rad3_minus_rad5^3 + 3/4*i_rad3_minus_rad5 + 2)*c],
  [(1/16*i_rad3_minus_rad5^3 - 3/4*i_rad3_minus_rad5 + 2)*c]],
 [[(1/16*i_rad3_minus_rad5^3 + 1/4*i_rad3_minus_rad5)*c],
  [(-1/16*i_rad3_minus_rad5^3 - 1/4*i_rad3_minus_rad5)*c]],
 [[(-1/16*i_rad3_minus_rad5^3 + 3/4*i_rad3_minus_rad5 - 2)*c],
  [(1/16*i_rad3_minus_rad5^3 - 3/4*i_rad3_minus_rad5 - 2)*c]],
 [[(-1/16*i_rad3_minus_rad5^3 + 3/4*i_rad3_minus_rad5)*c],
  [(1/16*i_rad3_minus_rad5^3 - 3/4*i_rad3_minus_rad5)*c]]]

In [61]:
i_rad3, rad5

(1/16*i_rad3_minus_rad5^3 + 1/4*i_rad3_minus_rad5,
 1/16*i_rad3_minus_rad5^3 - 3/4*i_rad3_minus_rad5)

In [62]:
# SUBFAMILIES OF SE4 WITH 10 ECKARDT POINTS

In [63]:
# CONDITIONS FOR SE6

In [64]:
Q6, table6 = get_unique_conditions(SE6)

In [65]:
for i in range(1,len(Q6)+1):
    print(Q6[i-1].factor(), " ", len([el[0] for el in table6 if el[1]==i])+6)

(5) * (c + (1/80*i_rad3_minus_rad5^3 - 3/20*i_rad3_minus_rad5)*e) * (c + (-1/80*i_rad3_minus_rad5^3 + 3/20*i_rad3_minus_rad5)*e)   10
(c + (1/16*i_rad3_minus_rad5^3 - 3/4*i_rad3_minus_rad5 - 2)*e) * (c + (-1/16*i_rad3_minus_rad5^3 + 3/4*i_rad3_minus_rad5 - 2)*e)   10
(3) * (c + (1/48*i_rad3_minus_rad5^3 + 1/12*i_rad3_minus_rad5)*e) * (c + (-1/48*i_rad3_minus_rad5^3 - 1/12*i_rad3_minus_rad5)*e)   18
(7) * (c + (1/112*i_rad3_minus_rad5^3 + 1/28*i_rad3_minus_rad5 + 2/7)*e) * (c + (-1/112*i_rad3_minus_rad5^3 - 1/28*i_rad3_minus_rad5 + 2/7)*e)   18
1   13


In [66]:
[[solve_linear_system_in_fraction_field([q[0]], [e], [c]) for q in Q6[i].factor()] for i in range(4)]

[[[(-1/16*i_rad3_minus_rad5^3 + 3/4*i_rad3_minus_rad5)*c],
  [(1/16*i_rad3_minus_rad5^3 - 3/4*i_rad3_minus_rad5)*c]],
 [[(-1/16*i_rad3_minus_rad5^3 + 3/4*i_rad3_minus_rad5 - 2)*c],
  [(1/16*i_rad3_minus_rad5^3 - 3/4*i_rad3_minus_rad5 - 2)*c]],
 [[(1/16*i_rad3_minus_rad5^3 + 1/4*i_rad3_minus_rad5)*c],
  [(-1/16*i_rad3_minus_rad5^3 - 1/4*i_rad3_minus_rad5)*c]],
 [[(1/16*i_rad3_minus_rad5^3 + 1/4*i_rad3_minus_rad5 - 2)*c],
  [(-1/16*i_rad3_minus_rad5^3 - 1/4*i_rad3_minus_rad5 - 2)*c]]]

In [67]:
i_rad3, rad5

(1/16*i_rad3_minus_rad5^3 + 1/4*i_rad3_minus_rad5,
 1/16*i_rad3_minus_rad5^3 - 3/4*i_rad3_minus_rad5)

In [68]:
# SUBFAMILIES OF SE6 WITH 10 ECKARDT POINTS

In [69]:
# SUBFAMILIES OF SE6 WITH 18 ECKARDT POINTS

In [70]:
# CONDITIONS FOR SE9

In [71]:
Q9, table9 = get_unique_conditions(SE9)

In [72]:
for i in range(1,len(Q9)+1):
    print(Q9[i-1].factor(), " ", len([el[0] for el in table9 if el[1]==i])+9)

(112) * (c + (1/112*i_rad3_minus_rad5^3 + 1/28*i_rad3_minus_rad5 - 2/7)*f)   18
(48) * (c + (-1/48*i_rad3_minus_rad5^3 - 1/12*i_rad3_minus_rad5)*f)   18
(112) * (c + (1/112*i_rad3_minus_rad5^3 + 1/28*i_rad3_minus_rad5 + 2/7)*f)   18
(16) * (c + (1/16*i_rad3_minus_rad5^3 + 1/4*i_rad3_minus_rad5)*f)   18


In [73]:
[solve_linear_system_in_fraction_field([poly], [f], [c]) for poly in Q9]

[[(1/16*i_rad3_minus_rad5^3 + 1/4*i_rad3_minus_rad5 + 2)*c],
 [(-1/16*i_rad3_minus_rad5^3 - 1/4*i_rad3_minus_rad5)*c],
 [(1/16*i_rad3_minus_rad5^3 + 1/4*i_rad3_minus_rad5 - 2)*c],
 [(1/48*i_rad3_minus_rad5^3 + 1/12*i_rad3_minus_rad5)*c]]

In [74]:
# SUBFAMILIES OF SE9 WITH 18 ECKARDT POINTS

In [75]:
i_rad3

1/16*i_rad3_minus_rad5^3 + 1/4*i_rad3_minus_rad5

In [76]:
rad5

1/16*i_rad3_minus_rad5^3 - 3/4*i_rad3_minus_rad5